In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install mplcursors

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install ipympl

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# import modules

import pandas as pd
import numpy as np
import openai
import tiktoken
import pyarrow

import matplotlib.pyplot as plt
import mplcursors

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


# notice that you will need to install certain packages to make this work (ipympl)
%matplotlib ipympl
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

# Initialize the GPT-3.5-turbo API client
# NOTE: Notebook may need to be restarted after you update the API key
#      for the new key to take effect. This is because a module once
#      loaded will not be reloaded even if you change the key in the
#      keys.py file. To force a reload, you must restart the notebook.


#openai.api_key = os.environ['sk-XXXXX']

In [7]:
df = pd.read_json('jira-conversations2.json')

In [8]:
import json

# Load the JSON data
with open("jira-conversations2.json", "r", encoding="utf-8") as json_file:
    chatbot_data = json.load(json_file)

In [9]:
len(chatbot_data)

4412

In [10]:
type(chatbot_data)

list

In [11]:
chatbot_data[1]

{'question000001': 'Hello, \n \n  \nI am ~+~+~+~+~+~ with ~+~+~+~+~+~ID - U20309912. \n  \nI have not received any mail regarding academic integrity course . \n\n  \nCan you please send the link for academic integrity course and details regarding it?! \n\n  \nThanks in advance. ~+~+~+~+~+~    \n',
 'response000002': 'Hello,\n~+~+~+~+~+~provide me with your USF email. \n',
 'question000003': 'Hello,     \n ~+~+~+~+~+~USF mail Id is [~~email~~3437635499608302290~~]   \n  \n Thanks in advance  \n ~+~+~+~+~+~ \n----\n \n {adf}{"type":"expand","content":[{"type":"paragraph","content":[{"type":"text","text":"Get "},{"type":"text","text":"Outlook for iOS","marks":[{"type":"link","attrs":{"href":"https://aka.ms/o0ukef"}}]}]}],"attrs":{"title":"Signature"}}{adf}\n',
 'response000004': 'Canvas invite has been sent via email. ',
 'question000005': 'Hello ,     \n I didn’t get any canvas invite to my mail.   \n  \n Can you please send it again?!   \n  \n Thanks in advance  \n ~+~+~+~+~+~ \n----\n 

### **DATA CLEANING USING REGULAR EXPRESSIONS**

In [12]:
import json
import re

# Define a function to clean the text in a given string
def clean_text(text):
    
    # Define a dictionary of contractions and their expanded forms
    contractions = {
        "can't": "can not",
        "won't": "will not",
        "i'm": "I am",
        "didn't": "did not",
        "couldn't": "could not",
        "isn't": "is not",
        "don't": "do not",
        "lil": "little",
        "alil": "a little",
        "i'll": "I will",
        "viaZ": " ",
    }

    # Create a regular expression pattern that matches the contractions
    pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b')

    # Replace contractions with their expanded forms
    text = pattern.sub(lambda match: contractions[match.group(0)], text)


    #Removing AM and PM
    text = re.sub(r'[AP]{1}M', ' ', text)

    # remove am or pm
    text = re.sub(r'\bam\b|\bpm\b', '', text)

    #Cleaning special characters
    text = re.sub(r'[&]+', ' ', text)

    #Cleaning special characters
    text = re.sub(r'[&]+', ' ', text)

    # Remove '~+~+~+~+~+~' placeholders
    text = text.replace('~+~+~+~+~+~', '')

    # Remove '[~~email~~3437635499608302290~~]' placeholders
    text = re.sub(r'\[~~email~~\d+~~\]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    # Remove *发件人:*
    text = text.replace('*发件人:*', '')

    # Remove URLs starting with "www."
    text = re.sub(r'www\.\S+', '', text)
    
    # Replace email addresses with an empty string
    text = re.sub(r'[\w\.-]+@[\w\.-]+\.[a-zA-Z]{1,}', '', text)


    # Remove characters like *_ *_, _, __, ___
    text = re.sub(r'[\*_]+', '', text)

    # Remove <[[email-...]]> patterns
    text = re.sub(r'<\[\[email-\d+\]\|mailto:\[email-\d+\]\]', '', text)

    # Remove [ [~~email~~...]|mailto:[~~email~~...]]
    text = re.sub(r'\[ \[~~email~~-\d+~~\]|mailto:\[~~email~~-\d+~~\]\]', '', text)

    # Remove [[~~email~~-...]|
    text = re.sub(r'\[\[~~email~~-\d+~~\]\|', '', text)

    # Remove additional patterns
    text = re.sub(r'\|\s*/\s*\[\s*\[~~email~~-\d+~~\]\s*\|.*?\]\s*|'
                  r'\[\S+\|https://\S+\]\s*', '', text)

    # Remove [https:// characters similar to this
    text = re.sub(r'\[https://', '', text)

    # Remove !image.png|thumbnail!
    text = re.sub(r'!\S+\|thumbnail!', '', text)

    # Remove characters similar to Sent from =X7aP0pxhHXZThDrV%2/SVKxDaxrhaIzarw7c9v07VxZSE%3D&reserved=0] for Windows
    text = re.sub(r'Sent from =[^]]*] for Windows', '', text)

    # Remove patterns similar to "Sent from [Outlook for Android|..."
    text = re.sub(r'Sent from \[.*?\|', '', text)

    ####
    text = re.sub(r'from\s?=.*reserved=0\s.*?Windows', '', text, flags=re.IGNORECASE)


    # Remove characters like '[' and '|'
    text = text.replace('[', '')
    text = text.replace('|', '')

    # Remove characters like '&', '?', '<>', ':', ',', and '\'
    text = text.replace('&', '')
    text = text.replace('?', '')
    text = text.replace('<>', '')
    text = text.replace(':', '')
    text = text.replace(',', '')
    text = text.replace('\\', '')

    # Remove characters similar to {adf}{"type":"expand","content":[{"type":"paragraph","content":[{"type":"text","text":"Get "},{"type":"text","text":"Outlook for iOS","marks":[{"type":"link","attrs":{"href":"https://aka.ms/o0ukef"}}]}]}],"attrs":{"title":"Signature"}}{adf}
    text = re.sub(r'{adf}{".*?"title":"Signature"}}{adf}', '', text)

    # Remove characters similar to [~~email~~-889629152828745785~~], [[~~email~~-4765935718946164845~~]|, and (0.0 kB)
    text = re.sub(r'\[~~email~~-\d+~~\]', '', text)
    text = re.sub(r'\[\[~~email~~-\d+~~\]\|', '', text)
    text = re.sub(r'\(0\.0 kB\)', '', text)

    # Remove [ |mailto:] or similar patterns
    text = re.sub(r'\[ \|.+?\]', '', text)

    # Remove hyperlinks starting with "https://"
    text = re.sub(r'https://\S+', '', text)

    # Remove '@' and '#' characters
    text = text.replace('@', '')
    text = text.replace('#', '')

    # To remove characters similar to —-—-—-,
    text = re.sub(r'—+', '', text)

    # To remove =
    text = text.replace("=", "")


    #replace consecutive spaces (1 to any number) with a single space
    text = re.sub(r' +', ' ', text)
    
    # Replace words starting with # with an empty string
    text = re.sub(r'#\w{1,}', '', text)

    # Remove characters similar to -1 using regex
    text = re.sub(r'-\d', '', text)

    # Remove contents similar to ~~email~~89629152828745785~~]
    text = re.sub(r'~~email~~\d+~~\]', '', text)


    # To remove characters similar to ~~email~~-889629152828745785~~]
    text = re.sub(r'~~email~~-\d+~~\]', '', text)

    # Remove angle brackets and their contents
    text = re.sub(r'<[^>]*>', '', text)

    # To remove characters similar to / and whitespaces
    text = re.sub(r'/', '', text)

    #Removing punctuations
    punctuations = '!"#$%&()*+,-./:;?@[]^_`{|}~'
    for p in punctuations:
        text = text.replace(p, " ")


    text = re.sub(r'\S*https\S*', '', text)



     # Remove content similar to {adf}{"type""expand""content"{"type""paragraph""content"{"type""text""text""Get "}{"type""text""text""Outlook for
    text = re.sub(r'\{adf\}\{"type".*?"text""Outlook for', '', text)

    # Remove the pattern
    text = re.sub(r'from =[\w\d%]+&reserved=\d+\s+for Windows', '', text)

    #To remove characters similar to ~accountid63c8b50ff6d42a7a4632e786]
    text = re.sub(r'~accountid[a-f0-9]+]', '', text)

    # To remove content similar to {adf}{"type":"expand","content":{"type":"paragraph","content":{"type":"text","text":"Get "},{"type":"text","text":"Outlook for
    text = re.sub(r'{adf}{"type":"expand","content":{"type":"paragraph","content":{"type":"text","text":"Get\s+"{"type":"text","text":"Outlook for.*?}}}}}', '', text)

    # Remove exclamation marks (!)
    text = text.replace('!', '')

    # Remove content similar to date/time formats
    text = re.sub(r'\b[MTWFS][a-z]+day\b [A-Z][a-z]+ \d{1,2} \d{4} \d{1,3} [APap][Mm]\b', '', text)

    # Remove content within parentheses and the parentheses
    text = re.sub(r'\(.*?\)', '', text)

    # Remove content similar to "---", "--", and "-"
    text = re.sub(r'-{1,3}', '', text)

    # Remove date-time patterns
    text = re.sub(r'\b(?:[A-Za-z]{3,} \d{1,2} \d{4} at \d{4})\b', '', text)

    # Remove content similar to "April 25th 2023"
    text = re.sub(r'\b[A-Z][a-z]+ \d{1,2}(st|nd|rd|th) \d{4}\b', '', text)

    # Remove content enclosed in parentheses
    text = re.sub(r'\([^)]*\)', '', text)

    # Remove empty strings
    #text = text.replace(' ', '')  # This will remove all spaces, making the string empty if it consists only of spaces

    # Remove commas
    text = text.replace(',', '')

    # Remove apostrophes
    text = text.replace("’", '')
    text = re.sub(r"[']", ' ', text)
    

    # Lowercasing
    text = text.lower()

    # Remove characters like (299 kB), (302 kB), (303 kB), and similar patterns
    text = re.sub(r'\(\d+ kb\)', '', text)

    # Remove characters similar to [~accountid:63c8b50ff6d42a7a4632e786]
    text = re.sub(r'\[\~accountid:[a-f0-9]+\]', '', text)

    # Remove months and weekdays
    text = re.sub(r"\b(?:january|february|march|april|may|june|july|august|september|october|november|december|"
                  r"jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|"
                  r"monday|tuesday|wednesday|thrusday|friday|saturday|sunday|mon|tue|wed|thr|thu|thru|thrus|fri|sat|sun)\b", '', text)
    
    # Remove ----
    text = text.replace('----', '')

    # Remove empty strings
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'^\s*$', '', text, flags=re.MULTILINE)

    return text

# Load the JSON data
with open("jira-conversations2.json", "r", encoding="utf-8") as json_file:
    chatbot_data = json.load(json_file)

# Iterate through the list of dictionaries and clean the text in each dictionary
for entry in chatbot_data:
    for key, value in entry.items():
        entry[key] = clean_text(value)

# Save the cleaned data back to the JSON file
with open("jira-conversations2-cleaned.json", "w", encoding="utf-8") as cleaned_json_file:
    json.dump(chatbot_data, cleaned_json_file, ensure_ascii=False, indent=4)

# Print a sample cleaned entry
print(chatbot_data[0]['question000001'])  # Access the first entry in the list



hii this is gadipally uid u6331114 in process of submitting ds160 i need to send address and phone number of point of contact school official so can you please send address and phone number of iris brito school official to contact uon arrival can i also know the first and last name of iris brito thank you


In [13]:
chatbot_data


[{'question000001': 'hii this is gadipally uid u6331114 in process of submitting ds160 i need to send address and phone number of point of contact school official so can you please send address and phone number of iris brito school official to contact uon arrival can i also know the first and last name of iris brito thank you',
  'response000002': 'thank you for reaching out you are able to add at best'},
 {'question000001': 'hello i with id u20309912 i have not received any mail regarding academic integrity course can you please send the link for academic integrity course and details regarding it thanks in advance',
  'response000002': 'hello provide me with your usf email',
  'question000003': 'hello usf mail id is thanks in advance adf type expand content type paragraph content type text text get type text text outlook for ios marks type link attrs href mso0ukef attrs title signature adf',
  'response000004': 'canvas invite has been sent via email',
  'question000005': 'hello i didn